# Single Peak Extraction

In this notebook, we first find valid subjects for extracting a single peak.
These are subjects where the number of peaks across channels is consistent,
the interbeat standard deviation is pretty low, and the heart rate is reasonable.

In [ ]:
import sys
sys.path.append('../')

from resnet import ResNet1d
import h5py
from itertools import product
import torch
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.signal import find_peaks
from scipy.stats import skew

In [ ]:
peak_prominence_for_detection = 0.75  # Minimum prominence required to retain a peak
inter_beat_sd_percentile = 0.90  # Percentile of standard deviations of inter beat SDs to set a cutoff for retention
mode_cutoff = 9  # Minimum number of channels with the same number of peaks required for retention
hr_low = 50  # Lowest HR for retention
hr_high = 120  # Highest HR for retention

In [ ]:
from constants import (
    DATA_DIR,
    N_LEADS,
)
config = '../model/config.json'

# Instantiate the model using the config.json information.
with open(config, 'r') as f:
    config_dict = json.load(f)
model = ResNet1d(
    input_dim=(N_LEADS, config_dict['seq_length']),
    blocks_dim=list(zip(config_dict['net_filter_size'], config_dict['net_seq_lengh'])),
    n_classes=1,
    kernel_size=config_dict['kernel_size'],
    dropout_rate=config_dict['dropout_rate']
)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Retrieve the state dict, which has all the coefficients
state_dict = (torch.load('../model/model.pth',
              weights_only=False,
              map_location=device))

# Load the state dict and set the model to eval mode.
model.load_state_dict(state_dict['model'])
model.eval()

# Read in exam metadata and limit to file 16.
df = pd.read_csv(f'{DATA_DIR}/exams.csv')
df = df[df['trace_file'] == 'exams_part16.hdf5']

# Read in raw ECG data for file 16.
filename = "../data/exams_part16.hdf5"

with h5py.File(filename, "r") as f:
    print("Keys in the HDF5 file:", list(f.keys()))
    dataset = f['tracings']
    print("Dataset shape:", dataset.shape)
    print("Dataset dtype:", dataset.dtype)
    data_array = f['tracings'][()]
    exam_ids = f['exam_id'][()]


In [ ]:
# Play around with different subjects and channels to see what peaks are detected.
subject = 4
channel = 0
series = data_array[subject, :, channel]
if skew(series) < 0:
    series = -series
peaks, _ = find_peaks(series, prominence=(peak_prominence_for_detection, None))
plt.plot(data_array[subject, :, channel])
plt.plot(peaks, data_array[subject, :, channel][peaks], "x")
plt.show()

In [ ]:
# For each subject and channel, calculate the number of peaks,
# the location of peaks,
# and the standard deviation of the interbeat intervals.
# This takes about 3 minutes.
summary_frame = pd.DataFrame(product(range(data_array.shape[0]), range(data_array.shape[2])),
                             columns=['subject', 'channel'])
summary_frame['n_peaks'] = np.nan
peak_list = []
summary_frame['inter_beat_sd'] = np.nan
for subject in range(data_array.shape[0]):
    if (subject + 1) % 1000 == 0:
        print('Subject ', subject + 1, ' of ', data_array.shape[0] + 1)
    for channel in range(12):
        series = data_array[subject, :, channel]
        if skew(series) < 0:
            series = -series
        peaks, _ = find_peaks(series, prominence=(peak_prominence_for_detection, None))
        n_peaks = len(peaks)
        inter_beat_sd = np.nan
        if n_peaks > 2:
            inter_beat_sd = np.std(np.diff(peaks))
        mask = (summary_frame['subject'] == subject) & (summary_frame['channel'] == channel)
        summary_frame.loc[mask, 'n_peaks'] = n_peaks
        summary_frame.loc[mask, 'inter_beat_sd'] = inter_beat_sd
        peak_list.append(peaks)
summary_frame['peaks'] = peak_list


In [ ]:
summary_frame.head()

In [ ]:
# For each subject, calculate the size of the mode of the number of peaks
modes = summary_frame.groupby('subject')['n_peaks'].value_counts().reset_index()
modes = modes.groupby('subject').first().reset_index()

# Find out what the most common mode is
modes['count'].value_counts()
# Having 4, 5, 6, 7, 8, 9, 10 channels with the same number of peaks is about
# equally common.

In [ ]:
modes.rename(columns={'n_peaks': 'mode_n_peaks',
                      'count': 'mode_count'}, inplace=True)
summary_frame = summary_frame.merge(modes[['subject', 'mode_n_peaks', 'mode_count']], how='inner', on='subject')
summary_frame.head()

In [ ]:
# Add the heart rate from the most common peak count
mask = summary_frame['n_peaks'] > 0
summary_frame.loc[mask, 'min_peak'] = summary_frame.loc[mask, 'peaks'].apply(np.nanmin)
summary_frame.loc[mask, 'max_peak'] = summary_frame.loc[mask, 'peaks'].apply(np.nanmax)
# Get the average number of observations between peaks
summary_frame['hr'] = (summary_frame['max_peak'] - summary_frame['min_peak']) / (summary_frame['n_peaks'] - 1)
# Heart rate is the number of beats per 10 seconds times 6.
summary_frame['hr'] = 6 * 4096 / summary_frame['hr']
# Only use the ones where the number of peaks matches the mode of n_peaks
temp = summary_frame[summary_frame['n_peaks'] == summary_frame['mode_n_peaks']].copy()
temp = temp.groupby('subject')['hr'].mean().reset_index()
# Drop the hr variable and merge back the subject-level summary variable.
summary_frame.drop(columns='hr', inplace=True)
summary_frame = summary_frame.merge(temp, on='subject', how='left')

In [ ]:
# Find the inter_beat_sd_percentile^th percentile of the inter_beat_sd
inter_beat_sd_cutoff = np.nanquantile(summary_frame['inter_beat_sd'], inter_beat_sd_percentile)

# For each subject, get the average inter beat sd using only the channels
# where the number of peaks is the mode number of peaks.
avg_inter_beat_sd = (
    summary_frame
    .loc[summary_frame['n_peaks'] == summary_frame['mode_n_peaks']]
    .groupby('subject')
    ['inter_beat_sd']
    .mean()
    .reset_index()
)
avg_inter_beat_sd.rename(columns={'inter_beat_sd': 'avg_inter_beat_sd'}, inplace=True)
summary_frame = summary_frame.merge(avg_inter_beat_sd, on='subject', how='left')

In [ ]:
# Flag subjects to retain if they have at least 9 channels with the same
# number of beats, an average inter_beat_sd on those channels less than the cutoff, 
# and a heart rate between 50 and 120
summary_frame['retain_subject'] = (
    (summary_frame['mode_count'] >= mode_cutoff)
    & (summary_frame['avg_inter_beat_sd'] < inter_beat_sd_cutoff)
    & ((summary_frame['hr'] >= hr_low) & (summary_frame['hr'] <= hr_high))
)


In [ ]:
summary_frame.head()

In [ ]:
# The number of subjects retained is the number of retain flags
# divided by the number of channels. This should be an integer or
# something has gone wrong.
np.sum(summary_frame['retain_subject']) / 12

In [ ]:
summary_frame.to_csv("../data/beats_summary_frame.csv", index=False)